In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity

/Users/davidpekar/Downloads/Python Projects/lyric generation/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("amishshah/song_lyrics")
dataset = dataset["train"].shuffle(seed=42)
subset_size = 10000
dataset = dataset.select(range(subset_size))
train_test_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_dataset["train"]
val_dataset = train_test_dataset["test"]
#train_test_dataset = dataset["train"].train_test_split(test_size=0.1)
#train_dataset = train_test_dataset["train"]
#val_dataset = train_test_dataset["test"]

In [3]:
#train_subset = train_dataset.select(range(100))
#val_subset = val_dataset.select(range(50))

In [56]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Ensure that tokenizer has padding token set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("amishshah/song_lyrics")
dataset = dataset["train"].shuffle(seed=42)
subset_size = 10000
dataset = dataset.select(range(subset_size))
train_test_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_dataset["train"]
val_dataset = train_test_dataset["test"]

# Load and prepare dataset
#dataset = load_dataset("amishshah/song_lyrics")
#train_test_dataset = dataset["train"].train_test_split(test_size=0.1)
#train_dataset = train_test_dataset["train"].select(range(100))
#val_dataset = train_test_dataset["test"].select(range(100))

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['lyrics'], truncation=True, padding=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
)

# Initialize Trainer
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

                                                    
  2%|▏         | 135/9000 [23:04<2:49:26,  1.15s/it] 

{'loss': 2.9365, 'grad_norm': 3.0167157649993896, 'learning_rate': 4.705882352941177e-05, 'epoch': 0.44}


                                                    
  2%|▏         | 135/9000 [41:58<2:49:26,  1.15s/it] 

{'loss': 2.808, 'grad_norm': 2.2655844688415527, 'learning_rate': 4.11764705882353e-05, 'epoch': 0.89}


                                                    
  2%|▏         | 135/9000 [1:00:59<2:49:26,  1.15s/it]

...

                                                      
  2%|▏         | 135/9000 [2:54:59<2:49:26,  1.15s/it]

{'loss': 2.547, 'grad_norm': 1.8551689386367798, 'learning_rate': 0.0, 'epoch': 4.0}


                                                      
100%|██████████| 9000/9000 [2:50:43<00:00,  1.14s/it]]

{'train_runtime': 10243.5585, 'train_samples_per_second': 3.514, 'train_steps_per_second': 0.879, 'train_loss': 2.6714083387586807, 'epoch': 4.0}


TrainOutput(global_step=9000, training_loss=2.6714083387586807, metrics={'train_runtime': 10243.5585, 'train_samples_per_second': 3.514, 'train_steps_per_second': 0.879, 'total_flos': 9406513152000000.0, 'train_loss': 2.6714083387586807, 'epoch': 4.0})

In [26]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline

# Define the path to the checkpoint you want to use
checkpoint_path = './results'

# Load the model and tokenizer from the checkpoint
model = GPT2LMHeadModel.from_pretrained(checkpoint_path)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_path)

# Ensure the tokenizer has a padding token set, if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Create the text generation pipeline using the model and tokenizer from the checkpoint
diomedes = pipeline('text-generation', model=model, tokenizer=tokenizer)
results = diomedes('Create a song about the magical feeling of a first kiss under the stars. ', max_length=600)
print(results[0]['generated_text'])


"""
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# Load the model and tokenizer for text generation
from transformers import pipeline

# Ensure your model and tokenizer are loaded correctly
diomedes = pipeline('text-generation', model='./results', tokenizer='./results')

# Generate text using the pipeline
results = diomedes('create a love song ', max_length=600)
print(results[0]['generated_text'])
"""

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Create a song about the magical feeling of a first kiss under the stars.                                                                                            A song like this tells your story at the end of the day.                                                     x 4

When I close my eyes and start singing
I see a magic mirror and my mind is screaming for you again
I am here again singing this song to remind you, the magic life, you will leave your feelings in I remember.                                   I am here again singing this song to remind you today, that you can leave them in I remember.                                                                    x 4
When I close my eyes
I see a magic mirror
I have never tried to forget
My magic mirror
Light
I look into your eyes and see you standing in the light
My magic mirror
Light
There’s nowhere I can be
I’m falling down
Dark inside of my mind
I feel such a great pain
I am falling down
Dark inside of my mind
I feel such a

"\nmodel.save_pretrained('./results')\ntokenizer.save_pretrained('./results')\n\n# Load the model and tokenizer for text generation\nfrom transformers import pipeline\n\n# Ensure your model and tokenizer are loaded correctly\ndiomedes = pipeline('text-generation', model='./results', tokenizer='./results')\n\n# Generate text using the pipeline\nresults = diomedes('create a love song ', max_length=600)\nprint(results[0]['generated_text'])\n"

In [51]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, DatasetDict

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Ensure that tokenizer has padding token set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Assuming the CSV file is at the path 'path_to_csv', and it includes 'tag' and 'lyrics' columns
dataset = load_dataset("amishshah/song_lyrics")
dataset = dataset["train"].shuffle(seed=42)
subset_size = 10000  # Adjust as necessary
dataset = dataset.select(range(subset_size))
train_test_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_dataset["train"]
val_dataset = train_test_dataset["test"]

def tokenize_function(examples):
    # Prepend the tag to each lyric in the batch
    concatenated_lyrics = ["[Genre: " + tag + "] " + lyric for tag, lyric in zip(examples["tag"], examples["lyrics"])]
    return tokenizer(concatenated_lyrics, truncation=True, padding="max_length", max_length=512)


train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

In [52]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

# Initialize Trainer
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

  1%|          | 100/9000 [01:52<2:45:14,  1.11s/it]

{'loss': 3.1795, 'grad_norm': 5.191074371337891, 'learning_rate': 1e-05, 'epoch': 0.04}


  2%|▏         | 200/9000 [03:44<2:43:59,  1.12s/it]

{'loss': 3.0335, 'grad_norm': 5.126195430755615, 'learning_rate': 2e-05, 'epoch': 0.09}


  3%|▎         | 300/9000 [05:36<2:42:10,  1.12s/it]

...

100%|██████████| 9000/9000 [2:50:50<00:00,  1.12s/it] 

{'loss': 2.4696, 'grad_norm': 1.7365777492523193, 'learning_rate': 0.0, 'epoch': 4.0}



100%|██████████| 9000/9000 [2:50:52<00:00,  1.14s/it] 

{'train_runtime': 10252.5589, 'train_samples_per_second': 3.511, 'train_steps_per_second': 0.878, 'train_loss': 2.635204077826606, 'epoch': 4.0}


TrainOutput(global_step=9000, training_loss=2.635204077826606, metrics={'train_runtime': 10252.5589, 'train_samples_per_second': 3.511, 'train_steps_per_second': 0.878, 'total_flos': 9406513152000000.0, 'train_loss': 2.635204077826606, 'epoch': 4.0})

In [51]:

# Save the model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# Load the model and tokenizer for text generation
from transformers import pipeline

# Ensure your model and tokenizer are loaded correctly
diomedes = pipeline('text-generation', model='./results', tokenizer='./results')

# Example of generating genre-specific text
genre = "rap"  # Replace with any genre present in your dataset
prompt = f"[Genre: {genre}] "
results = diomedes(prompt, max_length=600)
print(results[0]['generated_text'])

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[Genre: rap]  BEST ART IN THE WORLD [Art: rap]  YOU NEED TO BE HAVING THIS [art: rap]  IF I HAVE A LOT OF PEOPLE [art: rap] I'm gonna be the big one [art: rap] WHAT ARE YOU CALLING AN ECHOSETHER BITCH [art: rap] THIS RATE NEEDS TO BE SACRED WITH SHIT [art: rap] YAY [art: rap] NINE STREETS AVAILABLE IN BLACK ONLY [art: rap] I GOT NO TRUTH TO THAT, FUCKING KIND OF THIS [arts: rap] SICK OF MENTALLY CIVILIZED PEOPLE [arts: rap] Fucking FAGGED BITCHED IN A WOOF WHO DONT CANT CARE [art: rap] I AM NOT TRULY A FAG [art: rap] YOU COULD BITCH MY STEDGE TICKS [ART: rap] WHAT ARE YOU DOING HERE? [art: rap] WHAT A DEATH IS HUW? [art: rap] BITCHY STUCK IN A KID WITH THE FUCKING BITCH [art: rap] YOU SHOULD GET SAVED [art: rap] GET THE FUCK OUT [art: rap] YEAH [art: rap] YOU WANT A BITCH? [art: rap] YEAH [art: rap] DONT EVEN SCREAM [art: rap] YOU WONT SUE ME I WANT A FUCKING BITCH IN THE BUNNY [art: rap] HAAAAAAAAAAAAAAAAAAAS [art: rap] WOOOOOOOOOOOOOOOOOOOOOOOOO [art: rap] STOP FUCKING STUFF YOU BLOW

In [49]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline

# Define the path to the checkpoint you want to use
checkpoint_path = './results'

# Load the model and tokenizer from the checkpoint
model = GPT2LMHeadModel.from_pretrained(checkpoint_path)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_path)

# Ensure the tokenizer has a padding token set, if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Create the text generation pipeline using the model and tokenizer from the checkpoint
diomedes = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Now you can generate text using the pipeline with your fine-tuned model
genre = "pop"  # Replace with any genre present in your dataset
prompt = f"[Genre: {genre}] "  # Prompt includes a genre-specific tag
results = diomedes(prompt, max_length=600)
print(results[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[Genre: pop] 하어말를 야랥 덴언... It was a pretty cool video; the sound of the song went through a very special period as one would expect. It looks quite classic from the beginning of our recording (like a little song by "Wah! Yui!"). From the end of "I Like My Voice" onwards with "I Won't Stop Losing You" ("I Won't Do That"), the sounds of the song got more prominent. The song sounds like it was inspired by a Japanese song known as "Ima Wanna Go", which was written in the 1940s and in which an 8-inch tape of the same song was distributed. The "Wah!" song sounds like it could be very effective in a way for recording an old song, because as we mentioned, it was played from time to time - one day there may be a song that reminds one to "go" after another one, and this will lead one to thinking that "Go" was made by "Wah!" before the "Wah!" album came out. So "Wah!" is still a popular song from this era. The song is not only very popular because it has a very unusual sound and it's very very po

In [45]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline

# Load the pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Ensure the tokenizer has a padding token set, which is necessary for some configurations
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the model and tokenizer for text generation
diomedes = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Simple prompt
prompt = "Create a song about the magical feeling of a first kiss under the stars: "

# Generate text using the pipeline
results = diomedes(prompt, max_length=600)
print(results[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Write a song about the feeling of returning home after many years:  Happiness and happiness are usually associated with more than just the release of our spirits; they also occur in the form of a combination of sadness and pleasure. People do seem to have a tendency to want something back—happiness, energy, self-esteem, and joy—whether those things are in the form of a new record or not; these feelings (and their accompanying feelings of relief and relief, as well as feelings of being happy and relieved) are what allow our spirits to return to their original sense of home. The release of joy and happiness was seen in the form of singing a song about returning to that sense of home; it was not in isolation by the listeners: instead people seemed to be singing about a return to home they hadn't been able to come home with. Again, the way we approach the release of joy and happiness is similar to those that might occur on the release of sadness: we think of the emotion of returning to a s